In [1]:
import sys, os, re, copy
import dill as pickle # this serializes all the functions inside the quantification dict
import numpy as np
import scipy as sp
from natsort import natsorted
from scipy.optimize import newton, minimize, fsolve
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.colors as mcolors
import matplotlib.ticker as tck
from matplotlib.gridspec import GridSpec
palette = list(mcolors.TABLEAU_COLORS.keys())
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid", {
 'axes.spines.bottom': True,
 'axes.spines.left': True,
 'axes.spines.right': True,
 'axes.spines.top': True
})
sns.set(font_scale=1)
palette = list(mcolors.TABLEAU_COLORS.keys())
#sns.set_theme(style="ticks", palette="muted")
sns.set_style("whitegrid")
sns.set_theme(style="ticks", palette="muted")
sns.set_context("notebook", font_scale=1.5)

In [2]:
def impute_conc(piece_wise_fit_metab, response_ratio):
    '''
    This function imputes the concentration from a response ratio.
    '''
    response_ratio_range = np.array(list(piece_wise_fit_metab.keys()))
    mask_range = [response_ratio >= min_v and response_ratio <= max_v for max_v, min_v in response_ratio_range]
    k = tuple(response_ratio_range[mask_range][0])
    conc = piece_wise_fit_metab[k](response_ratio)
    estimator = 'intrapolation'
    if 0 in k:
        estimator = 'extrapolation under'
    elif np.inf in k:
        estimator = 'extrapolation over'
    return(conc, estimator)

# Calculate metabolite concentrations and merge with Asp-sensor signal

In [3]:
### Read quantification function ###
dict_pickle_fnam = 'AA_quant-dict.pickle'
with open(dict_pickle_fnam, 'rb') as handle:
    piece_wise_fit_quant = pickle.load(handle)

In [4]:
### Read measurements ###
### Replace all N/F with 0 before start ###
esheet_dict_mes = pd.read_excel('Asp-sens_steady-state_LCMS.xlsx', sheet_name=None)
annotation_df = pd.read_excel('Asp-sens_steady-state_LCMS-annotations.xlsx')
metab_dict_mes = dict()
metab_names_mes = list()
for k in esheet_dict_mes.keys():
    if 'U-13C' not in k:
        metab_names_mes.append(k)
        metab_dict_mes[k] = copy.deepcopy(esheet_dict_mes[k])
        metab_dict_mes[k]['Response Ratio'] = metab_dict_mes[k]['Area'] / metab_dict_mes[k]['ISTD Response']
        metab_dict_mes[k]['Response Ratio'] = metab_dict_mes[k]['Response Ratio'].fillna(0).replace(np.inf, 0)
        metab_dict_mes[k]['Sample_name'] = [fn.split('_')[-1] for fn in metab_dict_mes[k]['Filename']]
        # Add annotations:
        metab_dict_mes[k] = metab_dict_mes[k].merge(annotation_df, left_on='Filename', right_on='Filename')
        metab_dict_mes[k] = metab_dict_mes[k].drop(['Flag Details', 'Filename', 'Type', 'RT', 'Sample ID'], axis=1)

In [5]:
### Impute concentration and add to metabolite dataframe ###
rr_mes = dict() # for plotting 
imp_conc_mes = dict() # for plotting
for metab in metab_names_mes[:]:
    # Assign imputed concentration:
    if metab in piece_wise_fit_quant:
        conc_list = list()
        estimator_list = list()
        for rr in metab_dict_mes[metab]['Response Ratio'].values:
            conc, estimator = impute_conc(piece_wise_fit_quant[metab], rr)
            conc_list.append(conc)
            estimator_list.append(estimator)
        metab_dict_mes[metab]['imputed_sample_conc'] = conc_list
        metab_dict_mes[metab]['imputed_sample_conc'] = metab_dict_mes[metab]['imputed_sample_conc'] / metab_dict_mes[metab]['dilution']
        metab_dict_mes[metab]['imputed_sample_estimator'] = estimator_list

In [6]:
# Generate dataframe for concentrations and estimtors:
print_cols = ['Sample_name', 'Cell line', 'Drug', 'Conc (nM)', 'Rescue', 'IC_well', 'GFP', 'RFP', 'GFP/RFP']
conc_df = metab_dict_mes['Glutamine pos'].loc[:, print_cols]
est_df = metab_dict_mes['Glutamine pos'].loc[:, print_cols]

for metab_nam in metab_dict_mes:
    if 'imputed_sample_conc' in metab_dict_mes[metab_nam].columns:
        conc_df = conc_df.merge(metab_dict_mes[metab_nam].loc[:, ['Sample_name', 'imputed_sample_conc']], on='Sample_name')
        conc_df = conc_df.rename(columns={'imputed_sample_conc': metab_nam})
        est_df = est_df.merge(metab_dict_mes[metab_nam].loc[:, ['Sample_name', 'imputed_sample_estimator']], on='Sample_name')
        est_df = est_df.rename(columns={'imputed_sample_estimator': metab_nam})
    elif sum(metab_dict_mes[metab_nam]['Response Ratio']) > 0:
        conc_df = conc_df.merge(metab_dict_mes[metab_nam].loc[:, ['Sample_name', 'Response Ratio']], on='Sample_name')
        conc_df = conc_df.rename(columns={'Response Ratio': metab_nam})
        est_df[metab_nam] = 'Response Ratio'
    else:
        conc_df = conc_df.merge(metab_dict_mes[metab_nam].loc[:, ['Sample_name', 'Area']], on='Sample_name')
        conc_df = conc_df.rename(columns={'Area': metab_nam})
        est_df[metab_nam] = 'Peak area'

# Write to Excel file:
with pd.ExcelWriter('Asp-sens_steady-state_LCMS_conc.xlsx') as writer:  
    conc_df.to_excel(writer, sheet_name='metab')
    est_df.to_excel(writer, sheet_name='estimator')

# Plot relationship between LCMS measured metabolite concentration and Asp-sensor signal

In [7]:
### Sigmoidal fit of the Asp-sensor signal 
### as a function of absolute Asp concentration
def sig_fit(x, top, midpoint, bottom, slope):
    return(top + (bottom - top) / (1 + (x / midpoint)**slope))

def x_XX(top, midpoint, bottom, slope, XX):
    yXX = (top - bottom)*(100-XX)/100 + bottom
    mid = (midpoint * ((top - yXX) / (yXX - bottom))**(1.0 / slope))
    return(mid)

def sig_fit_loss(mes_y, x, p):
    pred_y = sig_fit(x, p[0], p[1], p[2], p[3])
    return(sum((mes_y-pred_y)**2))

def fit_sigmoid(x, y , percentiles, Ngrid, max_top):
    bnds = ((0, max_top), (0, None), (0, None), (0, None))
    def fun_sig_fit(p): return(sig_fit_loss(y, x, p))
    p_sig_fit = minimize(fun_sig_fit, (10, 0, 5, 1), method='L-BFGS-B', bounds=bnds)
    top, midpoint, bottom, slope = p_sig_fit.x

    x_lower = x_XX(top, midpoint, bottom, slope, percentiles[0])
    x_upper = x_XX(top, midpoint, bottom, slope, percentiles[1])
    x_sig = np.linspace(x_lower, x_upper, Ngrid)
    y_sig = sig_fit(x_sig, p_sig_fit.x[0], p_sig_fit.x[1], p_sig_fit.x[2], p_sig_fit.x[3])
    x50 = x_XX(top, midpoint, bottom, slope, 50)
    
    return(x_sig, y_sig, x50, bottom, top)

def plot_sigmoid(x, y, ax1, max_top=None, percentile=(1, 99)):
    x_sig, y_sig, x50, bottom, top = fit_sigmoid(x, y, percentile, 1000, max_top)
    ax1.plot(10**x_sig, y_sig)
    ax1.vlines(10**x50, bottom, top, ls='--', color='r')
    halfmax = round(10**x50)
    anno_str = 'Asp half-max\n{} $\mu$M'.format(halfmax)
    props = dict(boxstyle='round', facecolor='red', alpha=0.4)
    ax1.text(0.5, 0.4, anno_str, transform=ax1.transAxes, fontsize=14,
             verticalalignment='top', bbox=props)


In [8]:
conc_df['conc_res'] = ['{} + {}'.format(c, r) if r != 'None' else '{}'.format(c) for c, r in zip(conc_df['Conc (nM)'], conc_df['Rescue'])]

conc_res = list()
for c, r in zip(conc_df['Conc (nM)'], conc_df['Rescue']):
    if int(c) == c:
        c = int(c)
        
    if r == 'None':
        conc_res.append('{}'.format(c))
    else:
        if 'pyr' in r.lower():
            conc_res.append('{} + Pyr'.format(c, r))
        else:
            conc_res.append('{} + {}'.format(c, r))

conc_df['conc_res'] = conc_res

### Plots for 293t

In [9]:
mask = conc_df['Cell line'] == '293t Asp-sens-RFP'
plot_order = natsorted(conc_df.loc[mask, 'conc_res'])

metab_list = ['Aspartate neg', 'Asparagine pos', 'Glutamate neg', 'Glutamine pos', 'Rotenone pos']
with PdfPages('steady-state_plots/steady-state_293t_Asp-sens-RFP.pdf') as pp:
    for metab in metab_list:
        metab_nam = metab.split()[0]
        fig, ax1 = plt.subplots(1, 1, figsize=(4, 3))
        
        if metab_nam == 'Aspartate':
            y = conc_df.loc[mask, 'GFP/RFP']
            x = np.log10(conc_df.loc[mask, 'Aspartate neg'])
            plot_sigmoid(x, y, ax1, max_top=max(y)*1.2)
        else:
            g1 = sns.regplot(ax=ax1, data=conc_df[mask], y='GFP/RFP', x=metab, lowess=True, scatter_kws=dict(alpha=0))

        g2 = sns.scatterplot(ax=ax1, data=conc_df[mask], y='GFP/RFP', x=metab, hue='conc_res', zorder=10, alpha=0.8, edgecolor='black', linewidth=0.7, legend=True);
        g2.set_xscale('log', base=10)
        g2.set_ylabel('GFP/RFP (AU)')
        g2.set_xlabel('{} (µM)'.format(metab_nam))
        g2.set_title('293t Asp-sens-RFP')
        #g2.set_xticklabels(g1.get_xticklabels(), rotation=90, size=12)
        sns.move_legend(g2, "upper left", bbox_to_anchor=(1.01, 1), frameon=True, ncol=1, \
                        alignment='center', labelspacing=0.3, handletextpad=0.2, borderaxespad=0, \
                        handlelength=1, title='Rot (nM)')

        ax1.tick_params(which="both", bottom=True, labelrotation=25)
        pp.savefig(fig, bbox_inches='tight')
        plt.close(fig)

### Plots for HT1080

In [10]:
mask = conc_df['Cell line'] == 'HT1080 Nuc-RFP Asp-sens'
plot_order = natsorted(conc_df.loc[mask, 'conc_res'])

metab_list = ['Aspartate neg', 'Asparagine pos', 'Glutamate neg', 'Glutamine pos', 'Rotenone pos', 'Metformin pos']
with PdfPages('steady-state_plots/steady-state_HT1080_Nuc-RFP_Asp-sens.pdf') as pp:
    for metab in metab_list:
        metab_nam = metab.split()[0]
        for drug in ['rot', 'met', 'both']:
            fig, ax1 = plt.subplots(1, 1, figsize=(4, 3))
            mask_rot = mask & (conc_df['Drug'] == 'Rotenone')
            mask_met = mask & (conc_df['Drug'] == 'Metformin')

            if drug == 'both':
                if metab_nam == 'Aspartate':
                    y = conc_df.loc[mask, 'GFP/RFP']
                    x = np.log10(conc_df.loc[mask, 'Aspartate neg'])
                    plot_sigmoid(x, y, ax1, max_top=max(y)*1.2)
                else:
                    g1 = sns.regplot(ax=ax1, data=conc_df[mask], y='GFP/RFP', x=metab, lowess=True, scatter_kws=dict(alpha=0))
            elif drug == 'rot':
                if metab_nam == 'Aspartate':
                    y = conc_df.loc[mask_rot, 'GFP/RFP']
                    x = np.log10(conc_df.loc[mask_rot, 'Aspartate neg'])
                    plot_sigmoid(x, y, ax1, max_top=max(y)*1.2)
                else:
                    g1 = sns.regplot(ax=ax1, data=conc_df[mask_rot], y='GFP/RFP', x=metab, lowess=True, scatter_kws=dict(alpha=0))
            elif drug == 'met':
                if metab_nam == 'Aspartate':
                    y = conc_df.loc[mask_met, 'GFP/RFP']
                    x = np.log10(conc_df.loc[mask_met, 'Aspartate neg'])
                    plot_sigmoid(x, y, ax1, max_top=max(y)*1.2)
                else:
                    g1 = sns.regplot(ax=ax1, data=conc_df[mask_met], y='GFP/RFP', x=metab, lowess=True, scatter_kws=dict(alpha=0))

            if drug == 'rot' or drug == 'both':
                g2 = sns.scatterplot(ax=ax1, data=conc_df[mask_rot], y='GFP/RFP', x=metab, hue='conc_res', zorder=10, alpha=0.8, edgecolor='black', linewidth=0.7, legend=True, marker='X');
            if drug == 'met' or drug == 'both':
                g3 = sns.scatterplot(ax=ax1, data=conc_df[mask_met], y='GFP/RFP', x=metab, hue='conc_res', zorder=10, alpha=0.8, edgecolor='black', linewidth=0.7, legend=True, marker='d');


            g2.set_xscale('log', base=10)
            ax1.set_xscale('log', base=10)
            g2.set_ylabel('GFP/RFP (AU)')
            g2.set_xlabel('{} (µM)'.format(metab_nam))
            g2.set_title('HT1080 Nuc-RFP Asp-sens')
            #g2.set_xticklabels(g1.get_xticklabels(), rotation=90, size=12)

            if drug == 'both':
                sns.move_legend(g3, "upper left", bbox_to_anchor=(1.01, 1), frameon=True, ncol=2, \
                                alignment='center', labelspacing=0.3, handletextpad=0.2, borderaxespad=0, \
                                handlelength=1, title='Rot/Met (nM)')
            elif drug == 'rot':
                sns.move_legend(g2, "upper left", bbox_to_anchor=(1.01, 1), frameon=True, ncol=1, \
                                alignment='center', labelspacing=0.3, handletextpad=0.2, borderaxespad=0, \
                                handlelength=1, title='Rot (nM)')
            elif drug == 'met':
                sns.move_legend(g3, "upper left", bbox_to_anchor=(1.01, 1), frameon=True, ncol=1, \
                                alignment='center', labelspacing=0.3, handletextpad=0.2, borderaxespad=0, \
                                handlelength=1, title='Met (nM)')

            ax1.tick_params(which="both", bottom=True, labelrotation=25)

            pp.savefig(fig, bbox_inches='tight')
            plt.close(fig)

/Users/krdav/anaconda3/lib/python3.8/site-packages/statsmodels/nonparametric/smoothers_lowess.py:227: RuntimeWarning: invalid value encountered in divide
  res, _ = _lowess(y, x, x, np.ones_like(x),
<ipython-input-10-9ef3c385cbc7>:42: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  ax1.set_xscale('log', base=10)


### Plots for H1299

In [11]:
mask = (conc_df['Cell line'] == 'H1299 Nuc-RFP Asp-sens') & (conc_df['Drug'] != 'AOA') & (conc_df['Drug'] != 'Antimycin A')
plot_order = natsorted(conc_df.loc[mask, 'conc_res'])

metab_list = ['Aspartate neg', 'Asparagine pos', 'Glutamate neg', 'Glutamine pos', 'Rotenone pos', 'Metformin pos']
with PdfPages('steady-state_plots/steady-state_H1299_Nuc-RFP_Asp-sens.pdf') as pp:
    for metab in metab_list:
        metab_nam = metab.split()[0]
        for drug in ['rot', 'met', 'both']:
            fig, ax1 = plt.subplots(1, 1, figsize=(4, 3))
            mask_rot = mask & (conc_df['Drug'] == 'Rotenone')
            mask_met = mask & (conc_df['Drug'] == 'Metformin')

            if drug == 'both':
                if metab_nam == 'Aspartate':
                    y = conc_df.loc[mask, 'GFP/RFP']
                    x = np.log10(conc_df.loc[mask, 'Aspartate neg'])
                    plot_sigmoid(x, y, ax1, max_top=max(y)*1.2)
                else:
                    g1 = sns.regplot(ax=ax1, data=conc_df[mask], y='GFP/RFP', x=metab, lowess=True, scatter_kws=dict(alpha=0))
            elif drug == 'rot':
                if metab_nam == 'Aspartate':
                    y = conc_df.loc[mask_rot, 'GFP/RFP']
                    x = np.log10(conc_df.loc[mask_rot, 'Aspartate neg'])
                    plot_sigmoid(x, y, ax1, max_top=max(y)*1.2)
                else:
                    g1 = sns.regplot(ax=ax1, data=conc_df[mask_rot], y='GFP/RFP', x=metab, lowess=True, scatter_kws=dict(alpha=0))
            elif drug == 'met':
                if metab_nam == 'Aspartate':
                    y = conc_df.loc[mask_met, 'GFP/RFP']
                    x = np.log10(conc_df.loc[mask_met, 'Aspartate neg'])
                    plot_sigmoid(x, y, ax1, max_top=max(y)*1.2)
                else:
                    g1 = sns.regplot(ax=ax1, data=conc_df[mask_met], y='GFP/RFP', x=metab, lowess=True, scatter_kws=dict(alpha=0))

            if drug == 'rot' or drug == 'both':
                g2 = sns.scatterplot(ax=ax1, data=conc_df[mask_rot], y='GFP/RFP', x=metab, hue='conc_res', zorder=10, alpha=0.8, edgecolor='black', linewidth=0.7, legend=True, marker='X');
            if drug == 'met' or drug == 'both':
                g3 = sns.scatterplot(ax=ax1, data=conc_df[mask_met], y='GFP/RFP', x=metab, hue='conc_res', zorder=10, alpha=0.8, edgecolor='black', linewidth=0.7, legend=True, marker='d');


            g2.set_xscale('log', base=10)
            g2.set_ylabel('GFP/RFP (AU)')
            g2.set_xlabel('{} (µM)'.format(metab_nam))
            g2.set_title('H1299 Nuc-RFP Asp-sens')
            #g2.set_xticklabels(g1.get_xticklabels(), rotation=90, size=12)

            if drug == 'both':
                sns.move_legend(g3, "upper left", bbox_to_anchor=(1.01, 1), frameon=True, ncol=2, \
                                alignment='center', labelspacing=0.3, handletextpad=0.2, borderaxespad=0, \
                                handlelength=1, title='Rot/Met (nM)')
            elif drug == 'rot':
                sns.move_legend(g2, "upper left", bbox_to_anchor=(1.01, 1), frameon=True, ncol=1, \
                                alignment='center', labelspacing=0.3, handletextpad=0.2, borderaxespad=0, \
                                handlelength=1, title='Rot (nM)')
            elif drug == 'met':
                sns.move_legend(g3, "upper left", bbox_to_anchor=(1.01, 1), frameon=True, ncol=1, \
                                alignment='center', labelspacing=0.3, handletextpad=0.2, borderaxespad=0, \
                                handlelength=1, title='Met (nM)')

            ax1.tick_params(which="both", bottom=True, labelrotation=25)

            pp.savefig(fig, bbox_inches='tight')
            plt.close(fig)

/Users/krdav/anaconda3/lib/python3.8/site-packages/statsmodels/nonparametric/smoothers_lowess.py:227: RuntimeWarning: invalid value encountered in divide
  res, _ = _lowess(y, x, x, np.ones_like(x),


### Plots for H1299 GOT DKO

In [12]:
mask = conc_df['Cell line'] == 'H1299 GOT-DKO Nuc-RFP Asp-sens'
plot_order = natsorted(conc_df.loc[mask, 'conc_res'])

metab_list = ['Aspartate neg', 'Asparagine pos', 'Glutamate neg', 'Glutamine pos']
with PdfPages('steady-state_plots/steady-state_H1299-GOT-DKO_Nuc-RFP_Asp-sens.pdf') as pp:
    for metab in metab_list:
        metab_nam = metab.split()[0]
        fig, ax1 = plt.subplots(1, 1, figsize=(4, 3))
        
        
        if metab_nam == 'Aspartate':
            y = conc_df.loc[mask, 'GFP/RFP']
            x = np.log10(conc_df.loc[mask, 'Aspartate neg'])
            plot_sigmoid(x, y, ax1, max_top=max(y)*1.2)
        else: 
            g1 = sns.regplot(ax=ax1, data=conc_df[mask], y='GFP/RFP', x=metab, lowess=True, scatter_kws=dict(alpha=0))
        
        g2 = sns.scatterplot(ax=ax1, data=conc_df[mask], y='GFP/RFP', x=metab, hue='conc_res', zorder=10, alpha=0.8, edgecolor='black', linewidth=0.7, legend=True);
        g2.set_xscale('log', base=10)
        g2.set_ylabel('GFP/RFP (AU)')
        g2.set_xlabel('{} (µM)'.format(metab_nam))
        g2.set_title('H1299 GOT DKO Nuc-RFP Asp-sens')
        #g2.set_xticklabels(g1.get_xticklabels(), rotation=90, size=12)
        sns.move_legend(g2, "upper left", bbox_to_anchor=(1.01, 1), frameon=True, ncol=1, \
                        alignment='center', labelspacing=0.3, handletextpad=0.2, borderaxespad=0, \
                        handlelength=1, title='Media Asp (nM)')

        ax1.tick_params(which="both", bottom=True, labelrotation=25)
        pp.savefig(fig, bbox_inches='tight')
        plt.close(fig)